In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.spatial import distance_matrix
from math import ceil

##### Now, load the correlation file and export it to distance matrix

In [2]:
#corelate=pd.read_csv(r'boundary_corelation2.csv')
#corelate.set_index('pathid')
#y= pd.melt(corelate, id_vars= 'pathid', var_name=['c_name'], value_name='Value')

cf_list=pd.read_csv(r'cflist.csv')##read the file called "cflist"
cf_list.head(10)##show, and right now, the columns are unnamed, pathid, c_name and Value

,Unnamed: 0,pathid,c_name,Value
0,0,p1,p1,0.000
1,1,p2,p1,0.835
2,2,p3,p1,0.585
3,3,p4,p1,0.416
4,4,p5,p1,1.000
5,5,p6,p1,0.936
6,6,p7,p1,1.000
7,7,p8,p1,0.939
8,8,p9,p1,0.413
9,9,p10,p1,1.000


###### rename the Value column to CF, which is the value of the distance between two paths 

In [3]:
cf_list = cf_list.rename({'Value': 'CF'}, axis=1)
cf_list.head(10)

,Unnamed: 0,pathid,c_name,CF
0,0,p1,p1,0.000
1,1,p2,p1,0.835
2,2,p3,p1,0.585
3,3,p4,p1,0.416
4,4,p5,p1,1.000
5,5,p6,p1,0.936
6,6,p7,p1,1.000
7,7,p8,p1,0.939
8,8,p9,p1,0.413
9,9,p10,p1,1.000


##### - create a new column call "path_pair" - the reason to pair those paths is for creating the distance matrix
`which adds two paths together (all the unique paths + each unique id) so (62 paths vs path 1) first and so on`

In [4]:
cf_list['path pair']=cf_list['pathid'].astype(str)+ " "+ cf_list['c_name'].astype(str)
cf_list.head(10)

,Unnamed: 0,pathid,c_name,CF,path pair
0,0,p1,p1,0.000,p1 p1
1,1,p2,p1,0.835,p2 p1
2,2,p3,p1,0.585,p3 p1
3,3,p4,p1,0.416,p4 p1
4,4,p5,p1,1.000,p5 p1
5,5,p6,p1,0.936,p6 p1
6,6,p7,p1,1.000,p7 p1
7,7,p8,p1,0.939,p8 p1
8,8,p9,p1,0.413,p9 p1
9,9,p10,p1,1.000,p10 p1


In [5]:
print(cf_list['CF'].head(5))
# cf_list.head(5)

0    0.000
1    0.835
2    0.585
3    0.416
4    1.000
Name: CF, dtype: float64


##### Build Distance Matrix from the list of distance values 

In [6]:
# distance_matrix = cf_list.pivot(index=cf_list['pathid'],columns=cf_list['pathid'],values=[cf_list['Value'],cf_list['Value']])
index, column = 62,62
res=[]
k=0
if index*column !=len(cf_list['CF']):
    res = "Error Matching"
else:
    for i in range(0,index):
        sub=[]
        for j in range(0,column):
            sub.append(cf_list['CF'][k])
            k+=1
        res.append(sub)
# print(res)   
df_new=pd.DataFrame(res)
df_new=df_new.rename(columns=lambda x: f"path_id_{int(x)+1}")
df_new=df_new.rename(index=lambda x: f"path_id_{int(x)+1}")
df_new.to_csv('distance_matrix.csv')
# df_new.iloc[:0]
# df_new=df_new.add_prefix('path_id '+1)

In [7]:
print(df_new.head(5))

           path_id_1  path_id_2  path_id_3  path_id_4  path_id_5  path_id_6  \
path_id_1      0.000      0.835      0.585      0.416      1.000      0.936   
path_id_2      0.835      0.000      0.411      0.622      0.338      0.151   
path_id_3      0.585      0.411      0.000      0.372      0.589      0.506   
path_id_4      0.416      0.622      0.372      0.000      0.793      0.719   
path_id_5      1.000      0.338      0.589      0.793      0.000      0.325   

           path_id_7  path_id_8  path_id_9  path_id_10  ...  path_id_53  \
path_id_1      1.000      0.939      0.413       1.000  ...       0.940   
path_id_2      0.357      0.190      0.748       1.000  ...       0.466   
path_id_3      0.601      0.529      0.497       1.000  ...       0.797   
path_id_4      0.798      0.732      0.182       1.000  ...       0.940   
path_id_5      0.028      0.311      0.917       0.845  ...       0.622   

           path_id_54  path_id_55  path_id_56  path_id_57  path_id_58  \
p

<h2><font color=orange>Enhenced code based on Chintan's for analysing the errors and clusters</font></h2>

### read from different output generated by three clustering methods and rename data

In [8]:

def aggregation(col,label_data,frequent_pathid):
    ###### Loop for the number of "label_data" columns, which means each result with diff params
    for i in range(len(label_data.columns)):##data's column length is 12 atm 
        #join proportion_and_pathid and label_data from column 0 (initial value) to column number x (col) 
        merged_data = frequent_pathid.join(label_data.iloc[:,col::])
         #rename the third column name to 'cluster'
        merged_data.columns[2]
    #     merged_data.rename(columns={merged_data.columns[2]:'label_'}, inplace=True)
        #increase column number for the next loop
        col+=1
        ##create two empty arrays for storing the error data as well as the cluster data latter
        errors=[]
        cluster_id=[]
    ####################################################################
        # unique_clusters= merged_data['label_0.5_4'].unique()
        unique_clusters = merged_data.iloc[:,2].unique()##the column "with 'label_x_x'" from the dataset is in column index 2
        unique_clusters_update=[]
    #     print(unique_clusters)
        for i in unique_clusters:
            if i>-1:
                unique_clusters_update.append(i)
    ############################### about C1  ########################
        for cluster_number in unique_clusters_update:
        #     print(cluster_number)
        ##saving the paths that do not belong to noise into screening
            screening = merged_data[(merged_data.iloc[:,2])==cluster_number]
        ##dont understand why I need this proportion, and how is the proportion were culculated
            frequency = screening['frequency']
            path_id= screening['pathid']
    #         label=screening['cluster']
            all_paths=[path for path in path_id]## cool way to do the for loop and present it into an array
#             most_frequent_path=all_paths[0] ##is replaced by the code below, as the machine should culculate the max and return the pathid rather than set it to index 0
            most_frequent_path=screening[screening['frequency']==screening['frequency'].max()]['pathid'].values
#             split= str(most_frequent_path).split(",")## I dont think I need this cuz its already obj rather than str
            print(most_frequent_path)
            less_frequent_paths = [path for path in all_paths if path not in most_frequent_path]
            path_combinination = list(itertools.product(most_frequent_path, less_frequent_paths))
            updated_path_combinination = [' '.join(i) for i in path_combinination]

            non_rep_proportions=[]
            non_rep_cf=[]##not representative paths' correlation formula

            for path in less_frequent_paths:
                non_rep_proportions.append(screening[screening['pathid']==path]['frequency'].iloc[0])
            for path in updated_path_combinination:
                non_rep_cf.append(cf_list.loc[cf_list['path pair'] == path, 'CF'].iloc[0])
        ##frequency correlation = frequency * the distance of each path pair
            fre_correlation= [a*b for a,b in zip(non_rep_proportions, non_rep_cf)]
        #     for a,b in zip(non_rep_proportions, non_rep_cf): 
        #         print("a- ",a)
        #         print("b- ",b)

        ## I dont understand why error is culculated by the sumary of fre_correlation
            cluster_error=sum(fre_correlation)
            errors.append(cluster_error)
            cluster_id.append(cluster_number)
        # print(errors)
    #################################################################### 
        outputs=pd.DataFrame()
        outputs['error']= [err for err in errors]
        outputs['cluster_id']= [ID for ID in cluster_id]
        Total_errors= outputs['error'].sum()
    #     print(Total_errors)
    ############################### About C2 and C3  ##################################### 
        C2=[]
        Cm=[]## this is removed
        C3=[]
        C3_fre=[]
        for x,y in merged_data.iterrows():##y is needed as a second parameter as the iterrows required
            if((merged_data['frequency'][x])>0.02 and (merged_data.iloc[:,2][x])!=-1):## the paths used greater than 15% is considered as frequent used paths
                # and(merged_data.iloc[:,2][x])!=-1
                C2.append(merged_data.iloc[:,2][x])
                if((merged_data.iloc[:,2][x])==-1):
                    Cm.append("found")##the frequency larger than 0.015 but clustered into -1
                    print(Cm)
    ########################## Analysing the noise cluster=rarely used paths ####################### 
            elif((merged_data.iloc[:,2][x])==-1):
                    C3_fre.append(merged_data['frequency'][x])
                    C3.append(merged_data.iloc[:,2][x])
        C3_error = sum(C3_fre)

    ##=======Save the errors and total cluster number with each iteration====== 
        total_err=round(((Total_errors+C3_error)*100), 2)
        total_path=len(outputs)+len(Cm)
        total_error.append(total_err)
        total_paths.append(total_path)

    ##=========  just for printing out the result here===========
        print("==========",merged_data.columns[2],"==========")

        print('Total Error = E1 +E3 : ',total_err, "%")
        print('final paths',total_path)
        print("======================")


In [9]:
dbscan_data=pd.read_csv('../Trajectory data/first_step_DBSCAN.csv')
optics_default_data=pd.read_csv('../Trajectory data/first_step_OPTICS_default.csv')
optics_dbscan_data=pd.read_csv('../Trajectory data/first_step_OPTICS_dbscan.csv')
HDBSCAN_data=pd.read_csv('../Trajectory data/first_step_HDBSCAN.csv')
Kmeans_data=pd.read_csv('../Trajectory data/first_step_K-means.csv')
##=== create empty list to store results ===
dfs=[]
col=0
def pre_process(data):
    data=data.rename({'proportion':'frequency'},axis=1)
    data=data.rename({'path id': 'pathid'}, axis=1)
# ##=== extract necessary data columns ===
    frequent_pathid=data[['frequency', 'pathid']]
    frequent_pathid
# ##=== extract columns which contain string 'label' ===
    label_data=data.filter(regex='label')
    aggregation(col,label_data,frequent_pathid)

#========== DBSCAN ==========
df_comparison=pd.DataFrame()
epsilons= ['0.2','0.25','0.3','0.35','0.4','0.45']
min_support=[2,3,4]
Eps=[]
Min_sup=[]
total_paths=[]
total_error=[]
###call the pre_process function
pre_process(dbscan_data)## DBSCAN

for eps in epsilons:
    for mins in min_support:
        Eps.append(eps)
        Min_sup.append(mins)
df_comparison['Eps']=Eps
df_comparison['MinSup']=Min_sup
df_comparison['Total_Paths']=total_paths
df_comparison['Total_Error']=total_error
df_comparison.to_csv('The result of DBSCAN.csv')

#========== Optics default ==========
##=== create empty list to store results ===
df_comparison=pd.DataFrame()
min_support=[2,3,4]
Min_sup=[]
total_paths=[]
total_error=[]
####call the pre_process function
pre_process(optics_default_data)
for mins in min_support:
    Min_sup.append(mins)
df_comparison['MinSup']=Min_sup
df_comparison['Total_Paths']=total_paths
df_comparison['Total_Error']=total_error
df_comparison.to_csv('The result of OPTICS default.csv')
#========== Optics dbscan ==========
df_comparison=pd.DataFrame()
max_epsilons= [0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55]
min_support=[2,3,4]
Max_eps=[]
Min_sup=[]
total_paths=[]
total_error=[]
####call the pre_process function
pre_process(optics_dbscan_data)
for max_eps in max_epsilons:
    for mins in min_support:
        Max_eps.append(max_eps)
        Min_sup.append(mins)
df_comparison['Max_Eps']=Max_eps
df_comparison['MinSup']=Min_sup
df_comparison['Total_Paths']=total_paths
df_comparison['Total_Error']=total_error
df_comparison.to_csv('The result of OPTICS dbscan.csv')
#========== HDBSCAN ==========
df_comparison=pd.DataFrame()
min_cluster_size=[2,3,4,5,6,8,10]
Min_cluster_size=[]
total_paths=[]
total_error=[]
####call the pre_process function
pre_process(HDBSCAN_data)
for mins in min_cluster_size:
    Min_cluster_size.append(mins)
df_comparison['Min_cluster_size']=Min_cluster_size
df_comparison['Total_Paths']=total_paths
df_comparison['Total_Error']=total_error
df_comparison.to_csv('The result of HDBSCAN.csv')

#========== K-means ==========
df_comparison=pd.DataFrame()
num_clusters=[3,4,5]
Num_clusters=[]
total_paths=[]
total_error=[]
####call the pre_process function
pre_process(Kmeans_data)
for num in num_clusters:
    Num_clusters.append(num)
df_comparison['Min_cluster_size']=Num_clusters
df_comparison['Total_Paths']=total_paths
df_comparison['Total_Error']=total_error
df_comparison.to_csv('The result of Kmeans.csv')



['p1']
['p2']
['p3']
['p4']
['p5']
['p10']
['p23']
['p24' 'p55']
['p31' 'p53']
['p35' 'p37']
['p43' 'p44']
========== label_0.2_2 ==========
Total Error = E1 +E3 :  3.34 %
final paths 11
['p1']
['p2']
['p3']
['p4']
['p5']
========== label_0.2_3 ==========
Total Error = E1 +E3 :  4.4 %
final paths 5
['p1']
['p2']
['p3']
['p4']
['p5']
========== label_0.2_4 ==========
Total Error = E1 +E3 :  4.45 %
final paths 5
['p1']
['p2']
['p3']
['p4']
['p5']
['p10']
['p23']
['p24' 'p55']
['p31' 'p53']
['p43' 'p44']
========== label_0.25_2 ==========
Total Error = E1 +E3 :  3.27 %
final paths 10
['p1']
['p2']
['p3']
['p4']
['p5']
========== label_0.25_3 ==========
Total Error = E1 +E3 :  4.21 %
final paths 5
['p1']
['p2']
['p3']
['p4']
['p5']
========== label_0.25_4 ==========
Total Error = E1 +E3 :  4.21 %
final paths 5
['p1']
['p2']
['p3']
['p10']
['p21']
['p30' 'p31' 'p53']
========== label_0.3_2 ==========
Total Error = E1 +E3 :  11.94 %
final paths 6
['p1']
['p2']
['p3']
['p21']
['p30' 'p31' 'p5

## The code below is being recreated within the above codes

In [10]:
# #========== DBSCAN ==========
# df_comparison=pd.DataFrame()
# epsilons= ['0.2','0.25','0.3','0.35','0.4','0.45']
# min_support=[2,3,4]
# Eps=[]
# Min_sup=[]
# Total_paths=[]
# aggregation(label_data,frequent_pathid)
# for eps in epsilons:
#     for mins in min_support:
#         Eps.append(eps)
#         Min_sup.append(mins)
# df_comparison['Eps']=Eps
# df_comparison['MinSup']=Min_sup
# df_comparison['Total_Paths']=total_paths
# df_comparison['Total_Error']=total_error
# df_comparison.to_csv('The result of DBSCAN.csv')

In [11]:
# ##========== Optics default ==========
# df_comparison=pd.DataFrame()
# min_support=[2,3,4]
# Min_sup=[]
# Total_paths=[]
# aggregation(label_data,frequent_pathid)
# for mins in min_support:
#     Min_sup.append(mins)
# df_comparison['MinSup']=Min_sup
# df_comparison['Total_Paths']=total_paths
# df_comparison['Total_Error']=total_error
# df_comparison.to_csv('The result of OPTICS default.csv')

In [48]:
dbscan=pd.read_csv('The result of DBSCAN.csv').drop('Unnamed: 0',axis=1,inplace=False)
optics=pd.read_csv('The result of OPTICS default.csv').drop('Unnamed: 0',axis=1,inplace=False)
hdbscan=pd.read_csv('The result of HDBSCAN.csv').drop('Unnamed: 0',axis=1,inplace=False)
kmeans=pd.read_csv('The result of Kmeans.csv').drop('Unnamed: 0',axis=1,inplace=False)


In [49]:
dbscan

,Eps,MinSup,Total_Paths,Total_Error
0,0.20,2,11,3.34
1,0.20,3,5,4.40
2,0.20,4,5,4.45
3,0.25,2,10,3.27
4,0.25,3,5,4.21
5,0.25,4,5,4.21
6,0.30,2,6,11.94
7,0.30,3,5,12.31
8,0.30,4,3,12.78
9,0.35,2,3,21.69


In [50]:
optics

,MinSup,Total_Paths,Total_Error
0,2,16,1.38
1,3,6,3.56
2,4,5,3.91


In [51]:
hdbscan = hdbscan.rename({'Min_cluster_size': 'c_size'}, axis=1)


In [52]:
kmeans=kmeans.rename({'Min_cluster_size': 'c_size'}, axis=1)

In [53]:
from IPython.display import display_html 


In [54]:
df1_styler = dbscan.style.set_table_attributes("style='display:inline'").set_caption('DBSCAN')
df2_styler = optics.style.set_table_attributes("style='display:inline'").set_caption('OPTICS')
df3_styler = hdbscan.style.set_table_attributes("style='display:inline'").set_caption('HDBSCAN')
df4_styler = kmeans.style.set_table_attributes("style='display:inline'").set_caption('Kmeans')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_()+df4_styler._repr_html_(), raw=True)

,Eps,MinSup,Total_Paths,Total_Error
0,0.2,2,11,3.34
1,0.2,3,5,4.4
2,0.2,4,5,4.45
3,0.25,2,10,3.27
4,0.25,3,5,4.21
5,0.25,4,5,4.21
6,0.3,2,6,11.94
7,0.3,3,5,12.31
8,0.3,4,3,12.78
9,0.35,2,3,21.69
